# Buidling a dataset

We're going to fetch and clean some data about how people use search. This stuff is tracked in real time by our reporting stack, but we want a static representative dataset for reproducible analysis here.

In [ ]:
import os
from elasticsearch import Elasticsearch
import pandas as pd

We need to create a client in order to interact with the reporting indexes in elastic cloud, and we'll use a couple of secrets from our project's environment to get access. Vercel adds quotes around variables in the `.env` file, so we'll strip those out with a quick lambda function.

In [ ]:
secret = lambda name: os.environ[name][1:-1]

es = Elasticsearch(
    cloud_id=secret("ES_REPORTING_CLOUD_ID"),
    http_auth=(secret("ES_REPORTING_USER"), secret("ES_REPORTING_PASSWORD")),
)

We'll use our client to get data from the `conversion` index. That index tracks a lot of stuff, but all we're interested in are the searches, ie documents where a set of search terms exists in the `"page.query.query"` field.

We'll also restrict our query to a known date range, and only include searches from the `/works` endpoint.

In [ ]:
resp = es.search(
    index="conversion",
    body={
        "query": {
            "bool": {
                "filter": [
                    {
                        "exists": {
                            "field": "page.query.query",
                            "field": "properties.totalResults",
                        }
                    },
                    {
                        "range": {
                            "@timestamp": {"gte": "2021-01-01", "lt": "2021-02-01"}
                        }
                    },
                    {"term": {"page.name": "works"}},
                ],
            }
        },
    },
    _source=["page.query.query", "properties.totalResults", "@timestamp"],
    size=100_000,
    request_timeout=30,
)

Next we'll clean up the raw data from the elasticsearch response. Working with `pandas` makes this process quite neat and readable. 

We're flattening the data, adding some more readable column headings, and stripping out the times from the timestamps - we might want to know which query we were running when these searches were originally run, so the date is worth keeping, but we don't need the precise second. The date gives us what we need while minimising the risk of tying searches back to individuals.

In [ ]:
df = pd.DataFrame(
    [
        {
            "search_terms": hit["_source"]["page"]["query"]["query"],
            "n_results": hit["_source"]["properties"]["totalResults"],
            "date": hit["_source"]["@timestamp"],
        }
        for hit in resp["hits"]["hits"]
    ]
)

df["date"] = pd.to_datetime(df["date"]).dt.date

Finally, we'll dump our dataset into a csv which we can reuse later on.

In [ ]:
df.to_csv("./searches.csv", index=None)